In [1]:
from modelRoPE0 import *

device = torch.device('cuda')

context_length = 128
embed_dim = 128
n_head = 4
n_layer = 2
batch_size = 64*4

vocab = Vocab()
model = LLM(vocab, context_length, embed_dim, n_head, n_layer).to(device)
model = torch.compile(model)
model.load_state_dict(torch.load("harry_potter1R.pt"))

<All keys matched successfully>

# Generate

In [2]:
g = model.generate(torch.tensor(vocab.encode("Harry potter was"), dtype=torch.long)[None,:].to(device), 2000, temperature=1.0, top_k=None)
print(vocab.decode(g[0].tolist()))

Harry potter was slumped along the wall,ze flatly dust with a wr of Wilkie Dumbledore, which were half done, but he had been disguised and far from Hogwarts (Much more homework — more than another means will be watered for your chances upon wishing you’d be able to us there … an excellent law sweater. What happened to catch the Invisibility Cloak Arthur’s supply of it dozed off. … She’d be alone in front of Dudley’s when Winky said, that she and Hermione passed off into the library.”
“You are asking to call this ball,” said Fudge, cazedly in his hand. “She’s so true about you.”
“Nothing, nobody?” said Harry flatly.
“Not even dangerous, Harry, Dumbledore!”
The sun placed on a particularly small roll of parchment on top of his shorts and puffed on a boggart, “ ‘to T washing a headouts around the dark?”
“Well, dear, and they weren’t going to keep up tied at the Quidditch House. ’cause he’s not.”
“You hear what,” said Hermione, tears shiningppering into the darkness, “His powers now remove

# Test

In [3]:
books = HarryPotter('harry_potter.txt')
val_set = HarryPotterDataset(context_length, books, vocab, train=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last=False, pin_memory=True, num_workers=8)
nll = model.evaluate_loss(val_loader, device)
print(f"---------------------")
print(f"Validation NLL {f'{nll:.2f}':>6}")
print(f"---------------------")

Evaluating: 100%|██████████| 657/657 [01:24<00:00,  7.74it/s]

---------------------
Validation NLL   4.42
---------------------
